In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import os

In [2]:
path = "./data/"
books_df = pd.read_csv(path + "books_word2vec.csv", index_col="Unnamed: 0")
users_df = pd.read_csv(path + "users.csv")
rating_df = pd.read_csv(path + "train_ratings.csv")

In [3]:
print(books_df.columns)
print(users_df.columns)
print(rating_df.columns)

Index(['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher',
       'img_url', 'language', 'category', 'summary', 'img_path'],
      dtype='object')
Index(['user_id', 'location', 'age'], dtype='object')
Index(['user_id', 'isbn', 'rating'], dtype='object')


In [4]:
user_rating_df = pd.merge(users_df, rating_df, how='left', on='user_id')
data = pd.merge(user_rating_df, books_df, how='left', on='isbn')
data.head(3)

,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,8,"timmins, ontario, canada",NaN,0002005018,4.0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Juvenile Fiction'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,8,"timmins, ontario, canada",NaN,074322678X,4.0,Where You'll Find Me: And Other Stories,Ann Beattie,2002.0,Scribner,http://images.amazon.com/images/P/074322678X.0...,en,['Fiction'],"Now back in print, Ann Beattie&#39;s finest sh...",images/074322678X.01.THUMBZZZ.jpg
2,8,"timmins, ontario, canada",NaN,0887841740,2.0,The Middle Stories,Sheila Heti,2004.0,House of Anansi Press,http://images.amazon.com/images/P/0887841740.0...,NaN,NaN,NaN,images/0887841740.01.THUMBZZZ.jpg


In [5]:
print(len(data))
data['user_id'].nunique()

315084


68092

In [6]:
data['location'] = data['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

data['location_city'] = data['location'].apply(lambda x: x.split(',')[0].strip())
data['location_state'] = data['location'].apply(lambda x: x.split(',')[1].strip())
data['location_country'] = data['location'].apply(lambda x: x.split(',')[2].strip())

data = data.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
data = data.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

data.drop('location', axis=1, inplace=True)

/tmp/ipykernel_18720/322630745.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['location'] = data['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거


In [7]:
data.drop(columns = ["img_path","img_url"], inplace=True)

In [8]:
def fillAge(idx):
    global data
    d2 = data.drop(idx, axis=0)
    d2 = d2.groupby('isbn').aggregate({"age":np.average})
    d2['age'] = d2['age'].astype(int)
    
    avg_age = {}
    for i, isbn in data.loc[idx,'isbn'].items():
        try: 
            avg_age[i] = d2.loc[isbn, 'age']
        except:
            pass
        
    avg_age = pd.DataFrame(avg_age, index=[0]).T
    avg_age.rename(columns={0:"avgAge"}, inplace=True)
    return avg_age

In [9]:
age_na_idx = data[data['age'].isna()].index

In [10]:
a = fillAge(age_na_idx)
a

,avgAge
0,39
19,31
20,41
21,28
22,36
...,...
314282,36
314380,16
314402,41
314452,28


In [11]:
data.loc[a.index, 'age'] = a['avgAge'].apply(lambda x: x)
data.loc[age_na_idx].head(5)

,user_id,age,isbn,rating,book_title,book_author,year_of_publication,publisher,language,category,summary,location_city,location_state,location_country
0,8,39.0,0002005018,4.0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,en,['Juvenile Fiction'],"In a small town in Canada, Clara Callan reluct...",timmins,ontario,canada
1,8,NaN,074322678X,4.0,Where You'll Find Me: And Other Stories,Ann Beattie,2002.0,Scribner,en,['Fiction'],"Now back in print, Ann Beattie&#39;s finest sh...",timmins,ontario,canada
2,8,NaN,0887841740,2.0,The Middle Stories,Sheila Heti,2004.0,House of Anansi Press,NaN,NaN,NaN,timmins,ontario,canada
3,8,NaN,1552041778,2.0,Jane Doe,R. J. Kaiser,1999.0,Mira Books,NaN,NaN,NaN,timmins,ontario,canada
4,8,NaN,1567407781,6.0,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998.0,Brilliance Audio - Trade,NaN,NaN,NaN,timmins,ontario,canada


In [12]:
bcpy = books_df.copy(deep=True)

In [13]:
# import re
# bcpy.loc[bcpy[bcpy['category'].notnull()].index, 'category'] = bcpy[bcpy['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())

In [14]:
# bcpy['category'] = bcpy['category'].str.lower()

In [15]:
# category_df = pd.DataFrame(bcpy['category'].value_counts()).reset_index()
# category_df.columns = ['category','count']
# category_df.head()

In [16]:
# category_df2 = category_df[category_df['count']>=50]
# categories = category_df2['category']

In [17]:
# for category in categories:
#     bcpy.loc[bcpy[bcpy['category'].str.contains(category,na=False)].index,'category_high'] = category

In [18]:
# category_high_df = pd.DataFrame(bcpy['category_high'].value_counts()).reset_index()
# category_high_df.columns = ['category','count']

In [19]:
# bcpy['category_high'].fillna('others', inplace=True)

In [20]:
# bcpy['category_high'].isna().sum()

In [21]:
# bcpy.drop(columns=['category'], inplace=True)
# bcpy.rename(columns={'category_high':'category'}, inplace=True)

In [22]:
# bcpy['category'].value_counts()

In [23]:
# bcpy

user 데이터 age 처리한거로 바꾸기..

In [24]:
def make_user_csv():
    ucpy = users_df.copy(deep=True)
    t = data[['user_id','age']]
    idx = t['age'].dropna().index
    t = t.loc[idx]
    t = t.groupby('user_id').aggregate({"age":np.max}).squeeze()
    ucpy['user_id2'] = ucpy['user_id'].copy()
    a = ucpy[['user_id', 'user_id2']].set_index('user_id2').squeeze()
    a = a.map(t).reset_index()
    a.columns = ['user_id','age']
    ucpy['age'] = a['age']
    ucpy.drop(columns = 'user_id2', inplace=True)
    return ucpy

ucpy = make_user_csv()

In [25]:
books_df

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Juvenile Fiction'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['History'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Social Science'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,['House & Home'],A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,['Juvenile Fiction'],A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,['Performing Arts'],These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg


In [26]:
bcpy

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Juvenile Fiction'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['History'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Social Science'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,['House & Home'],A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,['Juvenile Fiction'],A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,['Performing Arts'],These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg


In [27]:
path = "./testdata/"
ucpy.to_csv(path + 'users.csv', index=False)
bcpy.to_csv(path + 'books.csv', index=False)

In [44]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL DEEPFM --EPOCHS 10 --BATCH_SIZE 1024 --LR 0.001

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: jq3219. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_145148-2x0zf1iz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sage-breeze-22
wandb: ⭐️ View project at https://wandb.ai/jq3219/my-test_recsys_competition1-project
wandb: 🚀 View run at https://wandb.ai/jq3219/my-test_recsys_competition1-project/runs/2x0zf1iz
--------------- DEEPFM Load Data ----------

In [40]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL FM --EPOCHS 5 --LR 0.001

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: jq3219. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_144504-1p4udvkw
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run trim-meadow-19
wandb: ⭐️ View project at https://wandb.ai/jq3219/my-test_recsys_competition1-project
wandb: 🚀 View run at https://wandb.ai/jq3219/my-test_recsys_competition1-project/runs/1p4udvkw
--------------- FM Load Data --------------

In [30]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL FFM --EPOCHS 30 --LR 0.001

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
--------------- FFM Load Data ---------------
--------------- FFM Train/Valid Split ---------------
--------------- INIT FFM ---------------
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: jq3219. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_135000-3l5xh8bf
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ethereal-resonance-13
wandb: ⭐️ View project at https://wandb.ai/jq3219/my-test_recsys_competition

In [54]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL DeepCoNN --LR 0.1 --EPOCHS 50

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: jq3219. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_151234-3hgrsnb6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run comfy-microwave-29
wandb: ⭐️ View project at https://wandb.ai/jq3219/my-test_recsys_competition1-project
wandb: 🚀 View run at https://wandb.ai/jq3219/my-test_recsys_competition1-project/runs/3hgrsnb6
--------------- DeepCoNN Load Data ----

In [55]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL NCF

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: jq3219. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_151907-pigebq7y
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run stellar-glade-30
wandb: ⭐️ View project at https://wandb.ai/jq3219/my-test_recsys_competition1-project
wandb: 🚀 View run at https://wandb.ai/jq3219/my-test_recsys_competition1-project/runs/pigebq7y
--------------- NCF Load Data -----------

In [56]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL WDN

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: jq3219. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_152015-1h0au7ys
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run soft-gorge-31
wandb: ⭐️ View project at https://wandb.ai/jq3219/my-test_recsys_competition1-project
wandb: 🚀 View run at https://wandb.ai/jq3219/my-test_recsys_competition1-project/runs/1h0au7ys
--------------- WDN Load Data --------------

In [57]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL DCN

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: jq3219. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_152117-3phxpkio
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run smart-grass-32
wandb: ⭐️ View project at https://wandb.ai/jq3219/my-test_recsys_competition1-project
wandb: 🚀 View run at https://wandb.ai/jq3219/my-test_recsys_competition1-project/runs/3phxpkio
--------------- DCN Load Data -------------

In [58]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL DEEPFM

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: jq3219. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_152221-34nq3oj4
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run winter-dawn-33
wandb: ⭐️ View project at https://wandb.ai/jq3219/my-test_recsys_competition1-project
wandb: 🚀 View run at https://wandb.ai/jq3219/my-test_recsys_competition1-project/runs/34nq3oj4
--------------- DEEPFM Load Data ----------

In [62]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL FM

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: recsys09lev1. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_155842-31gx9ze3
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run stilted-dawn-1
wandb: ⭐️ View project at https://wandb.ai/recsys09lev1/competition1
wandb: 🚀 View run at https://wandb.ai/recsys09lev1/competition1/runs/31gx9ze3
--------------- FM Load Data ---------------
--------------- FM Train/V

In [60]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL FFM

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: jq3219. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /opt/ml/input/code/wandb/run-20221029_152428-1g6qtqt8
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run dry-terrain-35
wandb: ⭐️ View project at https://wandb.ai/jq3219/my-test_recsys_competition1-project
wandb: 🚀 View run at https://wandb.ai/jq3219/my-test_recsys_competition1-project/runs/1g6qtqt8
--------------- FFM Load Data -------------

In [46]:
!python3 main.py -h

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
usage: main.py [-h] [--DATA_PATH DATA_PATH]
               [--MODEL {FM,FFM,NCF,WDN,DCN,CNN_FM,DeepCoNN,DEEPFM}]
               [--DATA_SHUFFLE DATA_SHUFFLE] [--TEST_SIZE TEST_SIZE]
               [--SEED SEED] [--BATCH_SIZE BATCH_SIZE] [--EPOCHS EPOCHS]
               [--LR LR] [--WEIGHT_DECAY WEIGHT_DECAY] [--DEVICE {cuda,cpu}]
               [--FM_EMBED_DIM FM_EMBED_DIM] [--FFM_EMBED_DIM FFM_EMBED_DIM]
               [--DEEPFM_EMBED_DIM DEEPFM_EMBED_DIM]
               [--NCF_EMBED_DIM NCF_EMBED_DIM] [--NCF_MLP_DIMS NCF_MLP_DIMS]
               [--NCF_DROPOUT NCF_DROPOUT] [--WDN_EMBED_DIM WDN_EMBED_DIM]
               [--WDN_MLP_DIMS WDN_MLP_DIMS] [--WDN_DROPOUT WDN_DROPOUT]
               [--DCN_EMBED_DIM DCN_EMBED_DIM] [--DCN_MLP_DIMS DCN_MLP_DIMS]

In [52]:
!python3 ensemble.py --ENSEMBLE_FILES 20221029_134942_FM,20221029_135408_FFM,20221029_143714_DeepCoNN --ENSEMBLE_STRATEGY WEIGHTED --ENSEMBLE_WEIGHT 0.2,0.3,0.5

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
